### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [2]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Western,Adventure,Drama,...,Romance,Family,Biography,Musical,Sci-Fi,Thriller,Adult,Mystery,News,Game-Show
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020
0,1,114508,8,1381006850,2013-10-05 21:00:50,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,2,208092,5,1586466072,2020-04-09 21:01:12,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,2,358273,9,1579057827,2020-01-15 03:10:27,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2,10039344,5,1578603053,2020-01-09 20:50:53,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2,6751668,9,1578955697,2020-01-13 22:48:17,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [2]:
def rank_sort_cut_movie(movies, reviews):
    '''
    INPUT:
     movies - the original cleaned dataset
     reviews - the original cleaned dataset
    OUTPUT:
     rank_movie - A newly created dataframe with movies that are sorted by 
                highest avg rating, mumber of reviews, last rating time
                and must have a min of 5 ratings
    '''
    # creating new columns
    avg_rating = reviews.groupby('movie_id')['rating'].mean()
    num_rating = reviews.groupby('movie_id')['rating'].count()
    last_rating = reviews.groupby('movie_id')['date'].max()
    
    # create a new dataframe with the above 3 columns
    df = pd.DataFrame({'avg_rating':avg_rating, 'num_rating':num_rating, 'last_rating':last_rating})
    
    # join this new dataframe with the movie names mapped
    movie_map = movies[['movie_id', 'movie', 'genre']]
    df = movie_map.set_index('movie_id').join(df)
    
    # sort all the info according the most popular orders
    sort_df = df.sort_values(['avg_rating', 'num_rating', 'last_rating'], ascending=False)
    
    # add a cut that only considering data has a min of 5 ratings 
    rank_sort_cut_movie = sort_df[sort_df['num_rating'] >=5]
    
    return rank_sort_cut_movie

In [5]:
rank_sort_cut_movie(movies, reviews).head()

,movie,genre,avg_rating,num_rating,last_rating
movie_id,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy|Horror,10.0,48,2016-08-14 17:16:50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,10.0,28,2016-01-08 00:44:43
6662050,Five Minutes (2017),Short|Comedy,10.0,22,2019-04-20 22:29:19
2737018,Selam (2013),Drama|Romance,10.0,10,2015-05-10 22:56:01
5804314,Let There Be Light (2017),Drama,10.0,7,2019-12-25 16:27:47


In [3]:
def popular_recommendations(user_id, n_top, ranked_movies):
    '''
    INPUT:
     user_id - the user_id of the individual you are making recommendations for
     n_top - an integer of the number recommendations you want back
    OUTPUT:
     top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    
    top_movies = list(ranked_movies['movie'][:n_top])
    
    return top_movies # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [4]:
# Put your solutions for each of the cases here
ranked_movies = rank_sort_cut_movie(movies, reviews)
# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations('1', 20, ranked_movies) # Your solution list here

# Top 5 movies recommended for id 53968|
recs_5_for_53968 = popular_recommendations('53968', 5, ranked_movies) # Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations('70000', 100, ranked_movies) # Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations('43', 35, ranked_movies) # Your solution list here



In [8]:
recs_20_for_1 = popular_recommendations('1', 20, ranked_movies)
recs_20_for_1

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Five Minutes (2017)',
 'Selam (2013)',
 'Let There Be Light (2017)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Crawl Bitch Crawl (2012)',
 'Chasing Happiness (2019)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Third Contact (2011)',
 'Romeo Juliet (2009)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Kitbull (2019)',
 'Agnelli (2017)',
 'Sátántangó (1994)',
 'Foster (2011)',
 'CM101MMXI Fundamentals (2013)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)']

In [35]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [29]:
ranked_movies[ranked_movies['genre'].str.contains('History|Drama') == True]

,movie,genre,avg_rating,num_rating,last_rating
movie_id,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy|Horror,10.000000,48,2016-08-14 17:16:50
2737018,Selam (2013),Drama|Romance,10.000000,10,2015-05-10 22:56:01
5804314,Let There Be Light (2017),Drama,10.000000,7,2019-12-25 16:27:47
4448444,Make Like a Dog (2015),Short|Comedy|Drama,10.000000,5,2017-09-09 13:51:48
2059318,Third Contact (2011),Drama|Mystery|Sci-Fi|Thriller,10.000000,5,2014-02-13 15:48:31
1431149,Romeo Juliet (2009),Drama,10.000000,5,2013-07-26 17:19:01
5512872,Be Somebody (2016),Comedy|Drama|Romance,9.985836,353,2018-05-17 00:29:18
4148400,Birlesen Gonuller (2014),Drama|History,9.882353,17,2015-10-22 23:34:37
9536832,Kitbull (2019),Animation|Short|Drama|Family,9.857143,7,2020-02-10 03:06:26


In [31]:
years=['2015']#['2015', '2016', '2017', '2018']
yrs = '|'.join(years)
yrs

'2015'

In [5]:
def popular_recomm_filter(user_id, n_top, ranked_movie, years=None, genres=None):
    '''
    INPUT:
     user_id - 
     n_top - 
     ranked_movie - 
     year - 
     genres -
    OUTPUT:
     top_movies - a list of the n_top recommended movies by movie titles in order
    '''
    
    if years is not None:
        yrs = '|'.join(years)
        ranked_movies[ranked_movies['last_rating'].str.contains(yrs) ==True]
    
    if genres is not None:
        gns = '|'.join(genres)
        ranked_movies[ranked_movies['genre'].str.find(gns) == True]
        
    top_movies = list(ranked_movies['movie'][:n_top])
    
    return top_movies

In [6]:
ranked_movies = rank_sort_cut_movie(movies, reviews)

# Top 20 movies recommended for id 1 with years=['2015', '2016', '2017', '2018'], genres=['History']
recs_20_for_1_filtered = popular_recomm_filter('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])

# Top 5 movies recommended for id 53968 with no genre filter but years=['2015', '2016', '2017', '2018']
recs_5_for_53968_filtered = popular_recomm_filter('53968', 5, ranked_movies, years=['2015', '2016', '2017', '2018'])

# Top 100 movies recommended for id 70000 with no year filter but genres=['History', 'News']
recs_100_for_70000_filtered = popular_recomm_filter('70000', 100, ranked_movies, genres=['History', 'News'])


In [7]:
print(recs_20_for_1_filtered)

['MSG 2 the Messenger (2015)', 'Avengers: Age of Ultron Parody (2015)', 'Five Minutes (2017)', 'Selam (2013)', 'Let There Be Light (2017)', "Quiet Riot: Well Now You're Here, There's No Way Back (2014)", 'Crawl Bitch Crawl (2012)', 'Chasing Happiness (2019)', 'Make Like a Dog (2015)', 'Pandorica (2016)', 'Third Contact (2011)', 'Romeo Juliet (2009)', 'Be Somebody (2016)', 'Birlesen Gonuller (2014)', 'Kitbull (2019)', 'Agnelli (2017)', 'Sátántangó (1994)', 'Foster (2011)', 'CM101MMXI Fundamentals (2013)', 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)']


In [37]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews)

# check 1 
assert t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History']) == recs_20_for_1_filtered,  "The first check failed..."
# check 2
assert t.popular_recs_filtered('53968', 5, ranked_movies, years=['2015', '2016', '2017', '2018']) == recs_5_for_53968_filtered,  "The second check failed..."
# check 3
assert t.popular_recs_filtered('70000', 100, ranked_movies, genres=['History', 'News']) == recs_100_for_70000_filtered,  "The third check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

AssertionError: The first check failed...